# Basic GEODYN Run

In [ ]:
%load_ext autoreload
%autoreload 2

import sys  
import pickle 
sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn

run_settings = '/data/zach_work/DRIA_CD_implementation/Run_cd_model_dria.yaml'

for i,val in enumerate(['dria_testA']):   #dria_testA
    run_settings = '/data/zach_work/DRIA_CD_implementation/jb2008_DRIAtests/Run_cd_model_'+ val+'.yaml'

    ### Load the data into an object
    RunObject = Pygeodyn(run_settings)
    RunObject.RUN_GEODYN()



In [ ]:
# sys.exit(0)

# Basic GEODYN Read

In [ ]:
%load_ext autoreload
%autoreload 2

import sys  
import pickle 
sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn

Obj_Geodyn={}

for i,val in enumerate(['dria_testA', 'control']):   #dria_testA
    run_settings = '/data/zach_work/DRIA_CD_implementation/jb2008_DRIAtests/Run_cd_model_'+ val+'.yaml'


    ### Load the data into an object
    Obj_Geodyn[val] = Pygeodyn(run_settings)
    Obj_Geodyn[val].getData()
    Obj_Geodyn[val].post_getData_ValidationMetrics()

# Plots

In [ ]:
import plotly.graph_objects as go
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd

import numpy as np

config = dict({
                'displayModeBar': True,
                'responsive': False,
                'staticPlot': False,
                'displaylogo': False,
                'showTips': False,
                })



import numpy as np
import pandas as pd
import os

# px.colors.colorscale_to_colors()
# plotly.colors.PLOTLY_SCALES["Viridis"]

def get_color(colorscale_name, loc):
    from _plotly_utils.basevalidators import ColorscaleValidator
    # first parameter: Name of the property being validated
    # second parameter: a string, doesn't really matter in our use case
    cv = ColorscaleValidator("colorscale", "")
    # colorscale will be a list of lists: [[loc1, "rgb1"], [loc2, "rgb2"], ...] 
    colorscale = cv.validate_coerce(colorscale_name)
    
    if hasattr(loc, "__iter__"):
        return [get_continuous_color(colorscale, x) for x in loc]
    return get_continuous_color(colorscale, loc)
        

# Identical to Adam's answer
import plotly.colors
from PIL import ImageColor

def get_continuous_color(colorscale, intermed):
    """
    Plotly continuous colorscales assign colors to the range [0, 1]. This function computes the intermediate
    color for any value in that range.

    Plotly doesn't make the colorscales directly accessible in a common format.
    Some are ready to use:
    
        colorscale = plotly.colors.PLOTLY_SCALES["Greens"]

    Others are just swatches that need to be constructed into a colorscale:

        viridis_colors, scale = plotly.colors.convert_colors_to_same_type(plotly.colors.sequential.Viridis)
        colorscale = plotly.colors.make_colorscale(viridis_colors, scale=scale)

    :param colorscale: A plotly continuous colorscale defined with RGB string colors.
    :param intermed: value in the range [0, 1]
    :return: color in rgb string format
    :rtype: str
    """
    if len(colorscale) < 1:
        raise ValueError("colorscale must have at least one color")

    hex_to_rgb = lambda c: "rgb" + str(ImageColor.getcolor(c, "RGB"))

    if intermed <= 0 or len(colorscale) == 1:
        c = colorscale[0][1]
        return c if c[0] != "#" else hex_to_rgb(c)
    if intermed >= 1:
        c = colorscale[-1][1]
        return c if c[0] != "#" else hex_to_rgb(c)

    for cutoff, color in colorscale:
        if intermed > cutoff:
            low_cutoff, low_color = cutoff, color
        else:
            high_cutoff, high_color = cutoff, color
            break

    if (low_color[0] == "#") or (high_color[0] == "#"):
        # some color scale names (such as cividis) returns:
        # [[loc1, "hex1"], [loc2, "hex2"], ...]
        low_color = hex_to_rgb(low_color)
        high_color = hex_to_rgb(high_color)

    return plotly.colors.find_intermediate_color(
        lowcolor=low_color,
        highcolor=high_color,
        intermed=((intermed - low_cutoff) / (high_cutoff - low_cutoff)),
        colortype="rgb",
    )




cols = get_color("Viridis", np.linspace(0, 1, 5))
map_cols = np.linspace(0, 1, 5)
colorscale=[]
for i,val in enumerate(map_cols):
    colorscale.append([val, cols[i]])
    



# Simplify Plotting Schemes:
col1 =  px.colors.qualitative.Plotly[0]
col2 =  px.colors.qualitative.Plotly[1]
col3 =  px.colors.qualitative.Plotly[2]
col4 =  px.colors.qualitative.Plotly[3]
col5 =  px.colors.qualitative.Plotly[4]
col6 =  px.colors.qualitative.Plotly[5]


In [ ]:
def legend_as_annotation(fig, den_model_string, color_it, x_annot, y_annot):
    fig.add_annotation(
            x=x_annot,
            y=y_annot,
            xref="paper",
            yref="paper",
            showarrow=False,
            text=den_model_string,
            font=dict(
                size=16,
                color="#ffffff"
                ),
            align="center",
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor=color_it,
            opacity=0.9
            )

    return(fig)

def add_stats_annotation(fig, text_in, col, x_annot, y_annot):
    fig.add_annotation(
            x=x_annot,
            y=y_annot,
            xref="x domain",
            yref="y domain",
            showarrow=False,
            text=text_in,
            font=dict(
                size=13,
                color="#ffffff"
                ),
            align="center",
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor=col,
            opacity=0.9
            )
    return(fig)

import pandas as pd

def STATS_residuals(residuals,measurement_type):
    import numpy as np
    n = np.size(residuals)
    mean = (1/n)*(np.sum(residuals))
    variance = (1/n)*(np.sum(np.square(residuals)))
    rms = np.sqrt(variance)
    rms_about_zero = np.sqrt((n/(n-1))*variance)
    
                
#     print('mean            ',measurement_type ,':',mean)
#     print('rms             ',measurement_type ,':',rms)
#     print('rms about zero  ',measurement_type ,':',rms_about_zero)
#     print()
    
    return(mean,rms,rms_about_zero)




def plotBasicResids_ChooseCoords(fig, obj_m1, coords, plotname, plot_num):

    if plot_num == 0:
        col = col1
        x_annot = x_annot_val
#         y_annot1 = 1
        y_annot2 = .9
        m_size = 3.5
    elif plot_num == 1:
        x_annot = 1.09
#         y_annot1 = .8
        y_annot2 = .75
        col = col2
        m_size = 3.5
    elif plot_num == 2:
        x_annot = 1.09
#         y_annot1 = .5
        y_annot2 = .5
        col = col3
        m_size = 3.5
    elif plot_num == 3:
        x_annot = 1.09
#         y_annot1 = .2
        y_annot2 = .253
        col = col4
        m_size = 3.5
    elif plot_num == 4:
        x_annot = 1.09
#         y_annot1 =  0 
        y_annot2 = .1
        col = col5
        m_size = 3.5
    elif plot_num == 5:
        x_annot = 1.09
#         y_annot1 =  0 
        y_annot2 = -.0810
        col = col6
        m_size = 3.5


    model_m1 = obj_m1.__dict__['global_params']['den_model']
    for ii,arc in enumerate(obj_m1.__dict__['global_params']['arc_input']):
        
        data_resids = obj_m1.__dict__['OrbitResids'][arc]['resids']


        fig.update_layout(title=''.join(coords)+' Orbit Residuals',
                        autosize=True,
                        font=dict(size=14),
                        legend= {'itemsizing': 'constant'})

        for i,val in enumerate(coords):
            fig.add_trace(go.Scattergl(
                                     x=data_resids['Date'][::50],
                                     y=data_resids[val][::50],
                                 name=val+' '+model_m1,
                                 mode='markers',
                                 marker=dict(
                                     color=col,
                                     size=m_size,
                                            ),
                                 showlegend=False),
                                 secondary_y=False,
                                 row=i+1, col=1)

#             (mean,rms,rms_about_zero) = STATS_residuals(data_resids[val], val+plotname)

#             fig = add_stats_annotation(fig, plotname+'<br>Mean='+ str(np.round(mean,4))+'<br>RMS='+ str(np.round(rms_about_zero,4)), col , x_annot, y_annot1)
    fig = legend_as_annotation(fig, plotname, col, x_annot, y_annot2)

        
    fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=1, col=1)
    fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=2, col=1)
    fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=3, col=1)
    fig.update_xaxes( title="Date",  row=3, col=1)

    return(fig)


def PLOT__intrack_residuals_w_rms_w_cd(fig, obj_m1, plot_name ,  plot_num):
    
    if plot_num == 0:
        col = col1
        x_annot = 1.09
        y_annot1 = 1
#         y_annot2 = .9
        m_size = 3.5
    elif plot_num == 1:
        x_annot = 1.09
        y_annot1 = .2
#         y_annot2 = .7
        col = col2
        m_size = 3.5
    elif plot_num == 2:
        x_annot = 1.09
        y_annot1 = .58
#         y_annot2 = .5
        col = col3
        m_size = 3.5
    elif plot_num == 3:
        x_annot = 1.09
        y_annot1 = .317
#         y_annot2 = .3
        col = col4
        m_size = 3.5
    elif plot_num == 4:
        x_annot = 1.09
        y_annot1 = .135
        col = col5
        m_size = 3.5
    elif plot_num == 5:
        x_annot = 1.09
        y_annot1 =  -0.05
        col = col6
        m_size = 3.5
    elif plot_num == 6:
        x_annot = 1.09
        y_annot1 =  -0.05
        col = col7
        m_size = 3.5
    elif plot_num == 7:
        x_annot = 1.09
        y_annot1 =  -0.05
        col = col8
        m_size = 3.5

    mean_tot = []     
    model_m1 = obj_m1.__dict__['global_params']['den_model']
    for ii,arc in enumerate(obj_m1.__dict__['global_params']['arc_input']):
        
        
        drag_data = obj_m1.__dict__['DragFile'][arc]
        data_resids = obj_m1.__dict__['OrbitResids'][arc]['resids']

        fig.add_trace(go.Scattergl(x=data_resids['Date'][::10],
                                   y=data_resids['T'][::10],
                                     #name= 'FracOx='+str(key[plot_name]['FracOx']).ljust(5)+' | a='+str(key[plot_name]['alpha']).ljust(8)+' | Kl='+str(key[plot_name]['KL']).ljust(5),
                                   name=plot_name,  
                                   mode='markers',
                                     opacity=0.8,
                                     marker=dict(
                                         color=col,
                                         size=m_size,),
                                     showlegend=True,
                                     ),
                                     secondary_y=False,
                                     row=1, col=1,
                                 )
        
        
        fig.add_trace(go.Scattergl(x=drag_data['Date'][:],
                                   y=drag_data['CD'][:],
                                     name= plot_name+arc,
                                     mode='markers',
                                     opacity=0.8,
                                     marker=dict(
                                         color=col,
                                         size=m_size,),
                                     showlegend=False,
                                     ),
                                     secondary_y=False,
                                     row=2, col=1,
                                 )
        fig.add_trace(go.Scattergl(x=drag_data['Date'][:],
                                   y=drag_data['TOTAREA'][:],
                                     name= plot_name+arc,
                                     mode='markers',
                                     opacity=0.8,
                                     marker=dict(
                                         color=col,
                                         size=m_size,),
                                     showlegend=False,
                                     ),
                                     secondary_y=False,
                                     row=3, col=1,
                                 )



#         (mean,rms,rms_about_zero) = STATS_residuals(data_resids['T'], 'in-track')
#         mean_tot.append(mean)                                                                                         
#     fig = add_stats_annotation(fig, plot_name +'<br>Mean='+ str(np.round(np.mean(mean_tot),4))+'<br>RMS='+ str(np.round(obj_m1.__dict__['Statistics']['T_RMS'].mean(),4)), col , x_annot, y_annot1)

    fig.update_yaxes( title="Residuals (m)",exponentformat= 'power',row=1, col=1)
    fig.update_yaxes( title="CD",exponentformat= 'power',row=2, col=1)
    fig.update_yaxes( title="Area (m^2)",exponentformat= 'power',row=3, col=1)
    fig.update_xaxes( title="Date", row=3, col=1)


    return(fig)



def PLOT__Tresids_w_cd(fig, obj_m1, plot_name , key, plot_num):
    
    if plot_num == 0:
        col = col1
        x_annot = 1.09
        y_annot1 = 1
#         y_annot2 = .9
        m_size = 3.5
    elif plot_num == 1:
        x_annot = 1.09
        y_annot1 = .2
#         y_annot2 = .7
        col = col2
        m_size = 3.5
    elif plot_num == 2:
        x_annot = 1.09
        y_annot1 = .58
#         y_annot2 = .5
        col = col3
        m_size = 3.5
    elif plot_num == 3:
        x_annot = 1.09
        y_annot1 = .317
#         y_annot2 = .3
        col = col4
        m_size = 3.5
    elif plot_num == 4:
        x_annot = 1.09
        y_annot1 = .135
        col = col5
        m_size = 3.5
    elif plot_num == 5:
        x_annot = 1.09
        y_annot1 =  -0.05
        col = col6
        m_size = 3.5
    elif plot_num == 6:
        x_annot = 1.09
        y_annot1 =  -0.05
        col = col7
        m_size = 3.5
    elif plot_num == 7:
        x_annot = 1.09
        y_annot1 =  -0.05
        col = col8
        m_size = 3.5

    mean_tot = []     
    model_m1 = obj_m1.__dict__['global_params']['den_model']
    for ii,arc in enumerate(obj_m1.__dict__['global_params']['arc_input']):
        
        
        drag_data = obj_m1.__dict__['DragFile'][arc]
        data_resids = obj_m1.__dict__['OrbitResids'][arc]['resids']

        
        fig.add_trace(go.Scattergl(x=data_resids['Date'][::10],
                                   y=data_resids['T'][::10],
                                     name= 'FracOx='+str(key[plot_name]['FracOx']).ljust(5)+' | a='+str(key[plot_name]['alpha']).ljust(8)+' | Kl='+str(key[plot_name]['KL']).ljust(5),
                                     mode='markers',
                                     opacity=0.8,
                                     marker=dict(
                                         color=col,
                                         size=m_size,),
                                     showlegend=True,
                                     ),
                                     secondary_y=False,
                                     row=1, col=1,
                                 )
        
        
        fig.add_trace(go.Scattergl(x=drag_data['Date'][:],
                                   y=drag_data['CD'][:],
                                     name= plot_name+arc,
                                     mode='markers',
                                     opacity=0.8,
                                     marker=dict(
                                         color=col,
                                         size=m_size,),
                                     showlegend=False,
                                     ),
                                     secondary_y=False,
                                     row=2, col=1,
                                 )
        fig.add_trace(go.Scattergl(x=drag_data['Date'][:],
                                   y=drag_data['TOTAREA'][:],
                                     name= plot_name+arc,
                                     mode='markers',
                                     opacity=0.8,
                                     marker=dict(
                                         color=col,
                                         size=m_size,),
                                     showlegend=False,
                                     ),
                                     secondary_y=False,
                                     row=3, col=1,
                                 )



#         (mean,rms,rms_about_zero) = STATS_residuals(data_resids['T'], 'in-track')
#         mean_tot.append(mean)                                                                                         
#     fig = add_stats_annotation(fig, plot_name +'<br>Mean='+ str(np.round(np.mean(mean_tot),4))+'<br>RMS='+ str(np.round(obj_m1.__dict__['Statistics']['T_RMS'].mean(),4)), col , x_annot, y_annot1)

    fig.update_yaxes( title="Residuals (m)",exponentformat= 'power',row=1, col=1)
    fig.update_yaxes( title="CD",exponentformat= 'power',row=2, col=1)
    fig.update_yaxes( title="Area (m^2)",exponentformat= 'power',row=3, col=1)
    fig.update_xaxes( title="Date", row=3, col=1)


    return(fig)




In [ ]:
Obj_Geodyn['dria_testA'].__dict__['DragFile']['2018.313']

In [ ]:
%load_ext autoreload
%autoreload 2


fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=(['In-Track Residuals', 'Drag Coefficient','Cross Sectional Area']),
    vertical_spacing = 0.1)

fig = PLOT__intrack_residuals_w_rms_w_cd(fig, Obj_Geodyn['control']    ,  'JB2008 BoxWing' , 0)
fig = PLOT__intrack_residuals_w_rms_w_cd(fig, Obj_Geodyn['dria_testA'] ,  'JB2008 DRIA'   , 1)

fig.update_layout(autosize=False,
    width=900,
    height=700)

fig.show(config=config)


